In [1]:
import polars as pl
from itertools import product
from functools import partial
from smaSingleParam import singlePickleTest, singleListTest, singleTest
from smaDoubleParam import doubleListTest, doublePickleTest, doubleTest
import multiprocessing

In [2]:
averages=[]
for k in range(10,1205,5):
    averages.append(pl.read_csv("../Data/sp500/sma/2017/Close/SimpleMovingAverages"+str(k)+".csv", infer_schema_length=None))

In [3]:
stocks=list(averages[0].columns)

In [4]:
smaList = list(range(0,6))
#afterList = list(range(5,16))
afterList = [5, 6, 10, 15]

In [5]:
#Single
params_combinations = list(product(smaList, smaList, afterList, afterList))
print(len(params_combinations))
k=0
for j in range(0,len(params_combinations)):
    if params_combinations[k][0] == params_combinations[k][1]:
        params_combinations.pop(k)
        k=k-1
    elif params_combinations[k][2] == params_combinations[k][3]:
        params_combinations.pop(k)
        k=k-1
    k=k+1
print(len(params_combinations))

params_combinations_short = params_combinations[300:310]

576
360


In [6]:
#Double
params_combinations = list(product(smaList, smaList, smaList, smaList, afterList, afterList))
print(len(params_combinations))
k=0
for j in range(0,len(params_combinations)):
    if params_combinations[k][0] == params_combinations[k][1]:
        params_combinations.pop(k)
        k=k-1
    elif params_combinations[k][2] == params_combinations[k][3]:
        params_combinations.pop(k)
        k=k-1
    elif params_combinations[k][4] == params_combinations[k][5]:
        params_combinations.pop(k)
        k=k-1
    elif params_combinations[k][0] == params_combinations[k][2]:
        if params_combinations[k][1] == params_combinations[k][3]:
            params_combinations.pop(k)
            k=k-1
    elif params_combinations[k][0] == params_combinations[k][3]:
        if params_combinations[k][1] == params_combinations[k][2]:
            params_combinations.pop(k)
            k=k-1
    k=k+1
print(len(params_combinations))

params_combinations_short = params_combinations[3200:3203]

20736
10080


In [14]:
passThroughTest = partial(singleTest, averages=averages, stocks=stocks)

In [13]:
%%timeit -n3
with multiprocessing.Pool(40) as pool:
    pool.map(
        passThroughTest,
        params_combinations_short
    )

6.87 s ± 67 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [11]:
%%timeit -n3
with multiprocessing.Pool(40) as pool:
    pool.map(
        singlePickleTest,
        params_combinations_short
    )

6.14 s ± 497 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [12]:
%%timeit -n3
with multiprocessing.Pool(40) as pool:
    pool.map(
        singleListTest,
        params_combinations_short
    )

4.29 s ± 77.6 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [6]:
with multiprocessing.Pool(40) as pool:
    pool.map(
        singleListTest,
        params_combinations
    )